In [45]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import random
 
from sklearn.model_selection import train_test_split
print('Versão do TensorFlow: ', tf.__version__)
tf.device('mps')

Versão do TensorFlow:  2.16.1


In [46]:
classes = ['RightRotate','LeftRotate','NoRotate']

def data_builder():
    inputs = []
    labels = []
    x = 0
    y = 0
    angle_max = 0
    for angle in range(-90, 90):
        if angle < 1:
            inputs.append([angle])
            labels.append([0])
        elif angle > 1:
            inputs.append([angle])
            labels.append([1])
    
    for angle in range(-90, 90):
            inputs.append([angle])
            labels.append([2])
    return inputs,labels


In [47]:
# Usando as funções
inputs, labels = data_builder()

# Convertendo para arrays numpy
inputs = np.array(inputs)
labels = np.array(labels)

# Split the data into training and testing sets
inputs_train, inputs_test, labels_train, labels_test = train_test_split(inputs, labels, test_size=0.1, random_state=42)

In [48]:
 
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=32, activation='relu', input_shape=(1,)),
    tf.keras.layers.Dense(units=64, activation='relu', ),
    tf.keras.layers.Dense(units=32, activation='relu',  ),
    tf.keras.layers.Dense(units=5, activation='softmax')
])

 

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [49]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [50]:
model.summary()
inputs = inputs.reshape(-1, 1)

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 32)             │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,421 (17.27 KB)

 Trainable params: 4,421 (17.27 KB)

 Non-trainable params: 0 (0.00 B)

In [54]:
%load_ext tensorboard
 

 
# Define TensorBoard callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1)

# Define CSVLogger callback
csv_logger = tf.keras.callbacks.CSVLogger('./logs/training.log')

# Train the model with TensorBoard and CSVLogger callbacks
model.fit(inputs_train, labels_train, epochs=2000, batch_size=32, validation_data=(inputs_test, labels_test), callbacks=[tensorboard_callback, csv_logger])

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Epoch 1/2000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5388 - loss: 0.6861 - val_accuracy: 0.3889 - val_loss: 0.7481
Epoch 2/2000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5178 - loss: 0.7014 - val_accuracy: 0.3889 - val_loss: 0.7256
Epoch 3/2000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5194 - loss: 0.6936 - val_accuracy: 0.4444 - val_loss: 0.7075
Epoch 4/2000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5295 - loss: 0.6940 - val_accuracy: 0.5000 - val_loss: 0.7159
Epoch 5/2000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5026 - loss: 0.6976 - val_accuracy: 0.4722 - val_loss: 0.7247
Epoch 6/2000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5129 - loss: 0.6925 - val_accuracy: 0.3889 - val_loss: 0.7154
Epoch 7/2000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4882 - loss: 0.6993 - val_accuracy: 0.4444 - val_loss: 0.7118
Epoch 8/2000
1

In [52]:
# Gerando novos dados de teste
test_inputs = np.array([[-80], [-50], [0], [30], [60]])
# Reshape dos dados de entrada para corresponder à expectativa do modelo
test_inputs = test_inputs.reshape(-1, 1)

# Fazendo previsões
predictions = model.predict(test_inputs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


In [53]:
# Decodificando as previsões
decoded_predictions = [np.argmax(prediction) for prediction in predictions]

# Imprimindo as previsões
for input_val, prediction in zip(test_inputs, decoded_predictions):
    print("Input:", input_val, "Prediction:", classes[prediction])

Input: [-80] Prediction: NoRotate
Input: [-50] Prediction: NoRotate
Input: [0] Prediction: RightRotate
Input: [30] Prediction: NoRotate
Input: [60] Prediction: NoRotate
